In [83]:
import pandas as pd
import torch
import torch.nn as nn
from tqdm import tqdm

In [84]:
###########################
# 1. 读取数据
###########################
input_csv = "NSL-KDD-Test.csv"
output_csv = "NSL-KDD-FGSM-Adversarial.csv"

df = pd.read_csv(input_csv)
print("原始数据条目数：", len(df))

# 假设最后一列是 class，其他列都是特征
feature_cols = df.columns[:-1]  # 除最后一列外
label_col = df.columns[-1]      # 最后一列

# 将特征和标签分开
X = df[feature_cols].copy()
y = df[label_col].copy()

原始数据条目数： 34394


In [85]:
###########################
# 2. 汇总数值特征与离散特征
###########################
# 在实际中可以根据 dtype，或手动指定
numeric_cols = ['duration', 'src_bytes', 'dst_bytes', 'wrong_fragment', 'urgent']
categorical_cols = [c for c in feature_cols if c not in numeric_cols]

# 处理数值特征
X_numeric = X[numeric_cols]

# 处理离散特征：将离散特征列转换为数值
X_categorical = X[categorical_cols]

# 将离散特征转换为数值（标签编码）
for col in X_categorical.columns:
    X_categorical.loc[:, col] = pd.factorize(X_categorical[col])[0]  # 使用 .loc 赋值

# 标签映射
label_map = {}
current_label_id = 0
for label_val in y.unique():
    label_map[label_val] = current_label_id
    current_label_id += 1

y_numerical = y.map(label_map)  # 将 y 转为 0, 1, 2,... 之类的int

In [86]:
###########################
# 3. 构造模型（数值特征与离散特征）
###########################
class SimpleNet(nn.Module):
    def __init__(self, numeric_input_dim, categorical_input_dim, embedding_dim=10, hidden_dim=32, output_dim=2):
        super(SimpleNet, self).__init__()
        
        # 数值特征的全连接层
        self.fc1_numeric = nn.Linear(numeric_input_dim, hidden_dim)
        
        # 离散特征的嵌入层
        self.embeddings = nn.ModuleList([
            nn.Embedding(len(X[c].unique()), embedding_dim) for c in categorical_cols
        ])
        
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim + len(categorical_cols) * embedding_dim, output_dim)

    def forward(self, numeric_input, categorical_input):
        # 数值特征处理
        numeric_out = self.relu(self.fc1_numeric(numeric_input))
        
        # 离散特征处理
        embedded = [embedding(categorical_input[:, i]) for i, embedding in enumerate(self.embeddings)]
        categorical_out = torch.cat(embedded, dim=1)
        
        # 合并数值特征与离散特征
        x = torch.cat([numeric_out, categorical_out], dim=1)
        out = self.fc2(x)
        return out

# 模型初始化
model = SimpleNet(numeric_input_dim=len(numeric_cols), categorical_input_dim=len(categorical_cols), embedding_dim=10)
model.eval()  # 这里仅为示例；真实要加载训练好的模型参数

loss_fn = nn.CrossEntropyLoss()
epsilon = 0.1  # 扰动系数，可自行调整

In [87]:
###########################
# 4. FGSM 函数（对数值特征和离散特征做扰动）
###########################
def fgsm_attack(model, loss_fn, numeric_data, categorical_data, label, epsilon):
    """
    对数值和离散特征做扰动
    numeric_data: 数值特征 tensor
    categorical_data: 离散特征 tensor
    label: [1] 的tensor, 真实或目标标签
    """
    # 前向传播
    output = model(numeric_data, categorical_data)
    loss = loss_fn(output, label)
    
    # 反向传播
    model.zero_grad()
    loss.backward()
    
    # data.grad 即为 dLoss/dData
    numeric_data_grad = numeric_data.grad.data
    categorical_data_grad = []
    
    # 对离散特征进行扰动
    for emb, cat_data in zip(model.embeddings, categorical_data.split(1, dim=1)):
        cat_data.requires_grad = True
        cat_data_grad = torch.autograd.grad(loss, cat_data)[0]
        categorical_data_grad.append(cat_data_grad)

    # 对数值特征和离散特征分别做扰动
    numeric_perturbed = numeric_data + epsilon * numeric_data_grad.sign()
    categorical_perturbed = torch.cat([cat_data + epsilon * cat_grad.sign() for cat_data, cat_grad in zip(categorical_data.split(1, dim=1), categorical_data_grad)], dim=1)

    return numeric_perturbed, categorical_perturbed

In [89]:
###########################
# 5. 对每一行做对抗扰动（数值特征和分类特征都做扰动）
###########################
X_numeric_adv = X_numeric.astype('float64').copy()  # 确保是 float 类型

# 对离散特征进行编码（确保它们是数值型）
X_categorical_encoded = X_categorical.apply(lambda col: pd.factorize(col)[0])

# 将离散特征转换为 Tensor
X_categorical_tensor = torch.tensor(X_categorical_encoded.values, dtype=torch.long)

# 创建进度条
for idx in tqdm(range(len(X_numeric)), desc="Generating adversarial samples"):
    # 1) 数值特征转 tensor
    row_values_numeric = X_numeric.iloc[idx].values.astype('float32')  # [input_dim, ]
    x_numeric = torch.tensor(row_values_numeric).unsqueeze(0)  # [1, input_dim]
    
    # 2) 离散特征转 tensor
    x_categorical = X_categorical_tensor[idx].unsqueeze(0)  # [1, num_categorical]
    
    # 对应标签
    label_val = y_numerical.iloc[idx]
    y_t = torch.tensor([label_val], dtype=torch.long)
    
    # 准备对 x 求梯度
    x_numeric.requires_grad = True
    x_categorical.requires_grad = False  # 离散特征不直接计算梯度，通过嵌入层进行
    
    # 执行 FGSM
    x_numeric_adv, x_categorical_adv = fgsm_attack(model, loss_fn, x_numeric, x_categorical, y_t, epsilon)
    
    # 将扰动后的数值列更新回
    X_numeric_adv.iloc[idx] = x_numeric_adv.detach().numpy().squeeze(0)

    # 对离散特征的处理：利用嵌入层并映射回类别
    # 让嵌入向量的梯度反向传播
    for i, emb in enumerate(model.embeddings):
        cat_data = x_categorical[:, i]  # 取当前类别的嵌入数据
        cat_data.requires_grad = True  # 让离散特征的嵌入层计算梯度
        emb_out = emb(cat_data)  # 获取嵌入向量
        
        # 使用 sign() 来做扰动
        emb_out_grad = emb_out.grad.data.sign() * epsilon
        x_categorical_adv[:, i] = emb_out + emb_out_grad
    
    # 将扰动后的离散特征映射回原类别（可以选择最近的嵌入）
    for i, col in enumerate(categorical_cols):
        max_idx = x_categorical_adv[0, i].argmax().item()
        X_categorical[col].iloc[idx] = X_categorical[col].unique()[max_idx]

Generating adversarial samples:   0%|          | 0/34394 [00:00<?, ?it/s]


RuntimeError: only Tensors of floating point and complex dtype can require gradients

In [ ]:
###########################
# 6. 合并离散特征 + 对抗后数值特征 + 标签
###########################
# 离散特征保持原样
X_adv = pd.concat([X_categorical, X_numeric_adv], axis=1)[feature_cols]

# 标签列保持原来的值
df_adv = pd.DataFrame(X_adv)
df_adv[label_col] = y  # class 列保持不变（或者写成 y.map(...) 也行）

# 保存
df_adv.to_csv(output_csv, index=False)
print(f"对抗样本已保存到 {output_csv}")

对抗样本已保存到 NSL-KDD-FGSM-Adversarial.csv
